In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# MODEL_DIR = "/data/jiangyy/phi-4/phi-4"

# # 加载tokenizer
# tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

# # 加载模型（确保模型以 .safetensors 格式保存）
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, trust_remote_code=True)
# model.eval()
# text = "This is an example sentence to test the model."
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# with torch.no_grad():
#     outputs = model(**inputs)

# logits = outputs.logits
# predicted_class = torch.argmax(logits, dim=1).item()

# print(f"Input text: {text}")
# print(f"Predicted class: {predicted_class}")


# from transformers import Qwen2ForCausalLM, Qwen2TokenizerFast

# model_name_or_path = "/data/jiangyy/Qwen2.5-Coder-7B-Instruct"
# tokenizer = Qwen2TokenizerFast.from_pretrained(model_name_or_path)
# print(tokenizer.get_chat_template())

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/jiangyy/lftrain2/LLaMA-Factory-main/saves/qwen25-7b/lora/sft2/checkpoint-2220"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "今天伦敦的天气怎么样"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]


tools = [
        {
            "name": "get_current_temperature",
            "description": "Get current temperature at a location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to get the temperature for."},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The unit to return the temperature in."}
                },
                "required": ["location"]
            }
        }
]

text = tokenizer.apply_chat_template(
    messages,
    tools=tools, 
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

model_name = "/data/jiangyy/Qwen2.5-Math-RM-72B"
device = "auto" # the device to load the model onto

model = AutoModel.from_pretrained(
        model_name, 
        device_map=device, 
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        ).eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

chat = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
        {"role": "assistant", "content": "To determine how much Janet makes from selling the duck eggs at the farmers' market, we need to follow these steps:\n\n1. Calculate the total number of eggs laid by the ducks each day.\n2. Determine how many eggs Janet eats and bakes for herself each day.\n3. Find out how many eggs are left to be sold.\n4. Calculate the revenue from selling the remaining eggs at $2 per egg.\n\nLet's start with the first step:\n\n1. Janet's ducks lay 16 eggs per day.\n\nNext, we calculate how many eggs Janet eats and bakes for herself each day:\n\n2. Janet eats 3 eggs for breakfast every morning.\n3. Janet bakes 4 eggs for her friends every day.\n\nSo, the total number of eggs Janet eats and bakes for herself each day is:\n\\[ 3 + 4 = 7 \\text{ eggs} \\]\n\nNow, we find out how many eggs are left to be sold:\n\\[ 16 - 7 = 9 \\text{ eggs} \\]\n\nFinally, we calculate the revenue from selling the remaining eggs at $2 per egg:\n\\[ 9 \\times 2 = 18 \\text{ dollars} \\]\n\nTherefore, Janet makes boxed18 dollars every day at the farmers' market."}] # 3.75

conversation_str = tokenizer.apply_chat_template(
                    chat, 
                    tokenize=False, 
                    add_generation_prompt=False
                    )

input_ids = tokenizer.encode(
            conversation_str, 
            return_tensors="pt", 
            add_special_tokens=False
            ).to(model.device)

outputs = model(input_ids=input_ids)
print(outputs[0])

In [16]:
from openai import OpenAI
import concurrent.futures

client = OpenAI(
    base_url="http://localhost:12000/v1",
    api_key="empty",
    # api_key=os.environ.get("OPENAI_API_KEY")
)

def call(prompt, question, history):
    models = client.models.list()
    model = models.data[0].id
    print(model)
    messages=[
        {"role": "system", "content": prompt}
    ]
    
    for item in history:
        messages.append({"role": "user", "content": list(item.keys())[0]})
        messages.append({"role": "assistant", "content": list(item.values())[0]})

    messages.append({"role": "user", "content": question})
    
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=1024,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        # extra_body={"chat_template_kwargs": {"enable_thinking": False}},
    )
    return completion.choices[0].message.content


# num_threads = 80
# while True:
#     with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#         futures = [executor.submit(call) for _ in range(num_threads)]
#         concurrent.futures.wait(futures)
        
msg = '''
###款项名称定义如下：
进度款-业务进度：与业务实施进度有关
进度款-时间进度：仅与固定时间有关
进度款-维保进度：与维护、维修有关
到货款：货物到货、签收
初验款：初验合格后
终验款：验收和终验都对应终验款
审计款：审计后
质保款：质保期满后

###合同内容
1.合同签订后30日内支付签约合同价85%的10%。
2.农民工工资款项进入农民工专户。
3.地下室顶板浇筑结束且基础验收完成后30日内,支付签约合同价85%的10%。
4.主体验收合格后30日内,支付签约合同价85%的10%。
5.所有系统调试完后30日内付签约合同价85%的10%。
6.工程竣工验收合格后30日内,工程款付至合同价的65%(含农民工工资)。
7.工程移交完整的计算资料后30日内,工程款付至合同价的70%(含农民工工资)。
8.工程结算经发包人委托的具有相应资质的造价咨询机构审定后后30日内,向分包人支付最终安装专业审定结算价的97%(提供全额发票)。
9.3%保修金按《工程质量保修书》的约定执行
①质量保修金为竣工结算价的3%
②发包人在工程竣工验收满2年后30天内付保修金的60%,满5年后30天内付保修金的40%。

请根据###合同内容，给出合同单次属于哪一类款项名称，同时计算出该合同单次的支付金额在整个签约合同价的占比，注意只需要百分比，不要给我金额数字，注意所有合同单次的占比总和为100%，注意 如果是“签约合同价85%的10%”，则占比为8.5%
'''

print(call("","hi",[]))


# nohup python continueReq.py >> continueReq.log 2>&1 & echo $! > continueReq.pid

/data1/jiangyy/Qwen3-14B/
<think>
Okay, the user said "hi". I need to respond appropriately. Since they're greeting me, I should greet them back and offer help. Maybe say something like "Hello! How can I assist you today?" That's friendly and open-ended. Let me check if there's any other context I should consider. The user hasn't provided any specific details, so keeping it general is best. Alright, that should work.
</think>

Hello! How can I assist you today? 😊


In [6]:
from openai import OpenAI
client = OpenAI(
    base_url="http://172.27.221.3:12000/v1",
    api_key="empty",
)
completion = client.chat.completions.create(
    model = client.models.list().data[0].id,
    messages=[{"role": "user", "content": "你是谁"}],
    extra_body={"stop_token_ids": [151329, 151336, 151338], "chat_template_kwargs": {"enable_thinking": False}},
    stream=True,
    temperature=0.7,
    max_tokens=1024,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        # print(chunk.choices[0].delta.content)
        print(chunk.choices[0].delta.content, end="")

我是通义千问，是阿里巴巴集团旗下的通义实验室研发的超大规模语言模型。我被设计用来回答各种问题、创作文字、进行逻辑推理、编程等任务。如果你有任何问题或需要帮助，欢迎随时告诉我！

In [17]:
import requests
import json

url = "http://localhost:12000/v1/chat/completions"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer empty"
}
data = {
    "messages": [
        {"role": "user", "content": "你好"}
    ],
    "model": "/data1/jiangyy/Qwen3-14B/",  # 注意：不能写绝对路径
    "stream": True
}

with requests.post(url, json=data, headers=headers, stream=True) as r:
    for line in r.iter_lines():
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                payload = line[len("data: "):]
                if payload.strip() == "[DONE]":
                    break
                data_json = json.loads(payload)
                delta = data_json["choices"][0]["delta"].get("content", "")
                print(delta, end="", flush=True)

<think>
好的，用户打招呼说“你好”，我需要友好回应。先确认是否有特殊需求，但看起来只是普通问候。保持自然，用中文回复，不用复杂结构。可以加个表情符号让语气更亲切。确保没有隐藏任务或问题，直接回应即可。
</think>

你好呀！😊 有什么我可以帮你的吗？

In [ ]:
import argparse
import pprint
import requests

def post_http_request(prompt: dict, api_url: str) -> requests.Response:
    headers = {"User-Agent": "Test Client"}
    response = requests.post(api_url, headers=headers, json=prompt)
    return response

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--host", type=str, default="localhost")
    parser.add_argument("--port", type=int, default=8081)
    parser.add_argument("--model",
                        type=str,
                        default="/data/jiangyy/Qwen2.5-Math-RM-72B")

    args = parser.parse_args()
    api_url = f"http://{args.host}:{args.port}/pooling"
    model_name = args.model

    # # Input like Completions API
    # prompt = {"model": model_name, "input": "vLLM is great!"}
    # pooling_response = post_http_request(prompt=prompt, api_url=api_url)
    # print("Pooling Response:")
    # pprint.pprint(pooling_response.json())

    # chat = [
    #         {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
    #             {"role": "user", "content": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
    #                 {"role": "assistant", "content": "To determine how much Janet makes from selling the duck eggs at the farmers' market, we need to follow these steps:\n\n1. Calculate the total number of eggs laid by the ducks each day.\n2. Determine how many eggs Janet eats and bakes for herself each day.\n3. Find out how many eggs are left to be sold.\n4. Calculate the revenue from selling the remaining eggs at $2 per egg.\n\nLet's start with the first step:\n\n1. Janet's ducks lay 16 eggs per day.\n\nNext, we calculate how many eggs Janet eats and bakes for herself each day:\n\n2. Janet eats 3 eggs for breakfast every morning.\n3. Janet bakes 4 eggs for her friends every day.\n\nSo, the total number of eggs Janet eats and bakes for herself each day is:\n\\[ 3 + 4 = 7 \\text{ eggs} \\]\n\nNow, we find out how many eggs are left to be sold:\n\\[ 16 - 7 = 9 \\text{ eggs} \\]\n\nFinally, we calculate the revenue from selling the remaining eggs at $2 per egg:\n\\[ 9 \\times 2 = 18 \\text{ dollars} \\]\n\nTherefore, Janet makes boxed18 dollars every day at the farmers' market."}] # 3.75


    # Input like Chat API
    prompt = {
        "model":
        model_name,
        "messages": [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
            }],
        }]
    }
    pooling_response = post_http_request(prompt=prompt, api_url=api_url)
    print("Pooling Response:")
    pprint.pprint(pooling_response.json())



# from openai import OpenAI

# # Modify OpenAI's API key and API base to use vLLM's API server.
# openai_api_key = "empty"
# openai_api_base = "http://localhost:8081/v1"

# client = OpenAI(
#     api_key=openai_api_key,
#     base_url=openai_api_base,
# )

# models = client.models.list()
# model = models.data[0].id

# responses = client.embeddings.create(
#     input="Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
#     model=model,
# )

# for data in responses.data:
#     print(data.embedding)

In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`

# from openai import OpenAI

# client = OpenAI(api_key="sk-4494bc7a7510462da2f58e52e5c90560", base_url="https://api.deepseek.com")

# question = '''
# There is a collection of $25$ indistinguishable white chips and $25$ indistinguishable black chips. Find the number of ways to place some of these chips in the $25$ unit cells of a $5\times5$ grid such that:

# each cell contains at most one chip
# all chips in the same row and all chips in the same column have the same colour
# any additional chip placed on the grid would violate one or more of the previous two conditions.
# '''

# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=[
#         {"role": "system", "content": "Your job is to solve the math problem step by step."},
#         {"role": "user", "content": question},
#     ],
#     stream=False
# )

# print(response.choices[0].message.content)

from openai import OpenAI
client = OpenAI(api_key="sk-4494bc7a7510462da2f58e52e5c90560", base_url="https://api.deepseek.com")

messages = [{"role": "user", "content": '1+2='}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

reasoning_content = ""
content = ""

for chunk in response:
    print(chunk)
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content
    else:
        content += chunk.choices[0].delta.content

In [ ]:
from openai import OpenAI
from time import sleep

client = OpenAI(
    base_url="https://172.27.221.3:3443/oscap/llm/d10/v1",  # 基础 URL，不包含 /chat/completions
    api_key="empty",  # 如果不需要认证，尝试用 "empty"，否则换成正确的 API Key
)


def call():
    response = client.chat.completions.create(
        model="d10",
        messages=[{"role": "user", "content": "帮我写个500字小作文"}],
        temperature=0.7,
        max_tokens=500
    )

while True:
    print(call())
    sleep(1)

In [ ]:
from openai import AzureOpenAI
#在playground里面验证成功的时候，点查看代码能看到下面的这4行。api_version靠猜是很难猜出来的
client = AzureOpenAI(
    api_key="187d20a319d4422198fc35ed62028e31",  
    api_version="2024-02-01",
    azure_endpoint="https://llmsecnew.openai.azure.com/"
)

chat_completion = client.chat.completions.create(
    model="gpt4turbo0409",      #这个地方是要你设定的deployment_name，不是具体的模型名称，也可以是基于gpt4创建的部署名
    messages=[        {
            "role": "user",
            "content": "x=3, x+y等于多少",
        },
        {
            "role": "assistant",
            "content": "请告诉我y是多少"
        },
        {
            "role": "user",
            "content": "y是100，那结果是多少"
        }],
    # stream=True,
    # extra_body={
    #     "data_sources": [
    #         {
    #             "type": "azure_search",
    #             "parameters": {
    #                 "endpoint": search_endpoint,
    #                 "index_name": search_index,
    #                 "authentication": {
    #                     "type": "system_assigned_managed_identity"
    #                 }
    #             }
    #         }
    #     ]
    # }
)

print(chat_completion.choices[0].message.content)

In [ ]:
# # # gpt-35-turbo-instruct
# from openai import AzureOpenAI

# client = AzureOpenAI(
#     api_key="e7d9310f4325408b8351712c27c7e4bb",
#     api_version="2024-02-15-preview",
#     azure_endpoint = "https://llmsec.openai.azure.com/"
#     )
    
# deployment_name="1030-1"#This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
    
# # Send a completion call to generate an answer
# print('Sending a test completion job')
# start_phrase = '美国的首都是什么'
# response = client.completions.create(model=deployment_name, prompt=start_phrase, max_tokens=10)
# print(start_phrase+response.choices[0].text)

### qwen api

https://help.aliyun.com/zh/dashscope/developer-reference/api-details#9c5fdab94csp4

### openai api
https://platform.openai.com/docs/api-reference/chat/create